In [1]:
from pycocotools.coco import COCO
import os
from tqdm import tqdm
import shutil

# COCO → YOLO dönüştürme (train / val / test ayrı)

In [2]:
# COCO JSON → YOLO formatına dönüştürür
from pycocotools.coco import COCO
import os
from tqdm import tqdm
import shutil

def convert_coco_to_yolo_flex(json_path, image_root_dirs, output_dir):
    labels_dir = os.path.join(output_dir, "labels")
    images_out_dir = os.path.join(output_dir, "images")

    os.makedirs(labels_dir, exist_ok=True)
    os.makedirs(images_out_dir, exist_ok=True)

    coco = COCO(json_path)

    categories = coco.loadCats(coco.getCatIds())
    class_id_map = {cat['id']: idx for idx, cat in enumerate(categories)}
    class_names = [cat['name'] for cat in categories]

    with open(os.path.join(output_dir, 'classes.txt'), 'w') as f:
        f.write('\n'.join(class_names))

    missing = 0
    found = 0

    for img_id in tqdm(coco.getImgIds(), desc=os.path.basename(json_path)):
        img = coco.loadImgs(img_id)[0]
        file_name = os.path.basename(img['file_name'])
        width = img['width']
        height = img['height']

        # Görseli bul
        found_path = None
        for root_dir in image_root_dirs:
            for root, _, files in os.walk(root_dir):
                if file_name in files:
                    found_path = os.path.join(root, file_name)
                    break
            if found_path:
                break

        if not found_path:
            missing += 1
            continue
        else:
            found += 1
            shutil.copyfile(found_path, os.path.join(images_out_dir, file_name))

        # Annotation'ı kaydet
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)
        yolo_lines = []
        for ann in anns:
            x, y, w, h = ann['bbox']
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w /= width
            h /= height
            class_id = class_id_map[ann['category_id']]
            yolo_lines.append(f"{class_id} {x_center} {y_center} {w} {h}")

        label_path = os.path.join(labels_dir, os.path.splitext(file_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            f.write('\n'.join(yolo_lines))

    print(f"Eşleşen: {found} — Eksik görseller: {missing}")


In [4]:
base_dir = r"C:\Users\hp\Desktop\YOLOv11_union_100"
image_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

convert_coco_to_yolo_flex(
    json_path=os.path.join(base_dir, "merged_annotations_cleaned_train.json"),
    image_root_dirs=image_dirs,
    output_dir=os.path.join(base_dir, "yolo_union_train")
)

convert_coco_to_yolo_flex(
    json_path=os.path.join(base_dir, "merged_annotations_cleaned_val.json"),
    image_root_dirs=image_dirs,
    output_dir=os.path.join(base_dir, "yolo_union_val")
)

convert_coco_to_yolo_flex(
    json_path=os.path.join(base_dir, "merged_annotations_cleaned_test.json"),
    image_root_dirs=image_dirs,
    output_dir=os.path.join(base_dir, "yolo_union_test")
)



loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


merged_annotations_cleaned_train.json: 100%|██████████| 1070/1070 [00:20<00:00, 52.37it/s]


Eşleşen: 1070 — Eksik görseller: 0
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


merged_annotations_cleaned_val.json: 100%|██████████| 356/356 [00:07<00:00, 48.95it/s]


Eşleşen: 356 — Eksik görseller: 0
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


merged_annotations_cleaned_test.json: 100%|██████████| 358/358 [00:07<00:00, 50.17it/s]

Eşleşen: 358 — Eksik görseller: 0


# YOLO Dataset Yapılandırması ve dataset.yaml Dosyasının Oluşturulması

In [5]:
def create_dataset_yaml(train_dir, val_dir, output_yaml_path):
    with open(os.path.join(train_dir, "classes.txt"), 'r') as f:
        class_names = [line.strip() for line in f.readlines()]

    lines = [
        f"path: ",
        f"train: {train_dir}/images",
        f"val: {val_dir}/images",
        "",
        "names:"
    ] + [f"  {i}: {name}" for i, name in enumerate(class_names)]

    with open(output_yaml_path, 'w') as f:
        f.write('\n'.join(lines))

    print(f"✅ YAML dosyası oluşturuldu: {output_yaml_path}")


In [6]:
create_dataset_yaml(
    train_dir=os.path.join(base_dir, "yolo_union_train"),
    val_dir=os.path.join(base_dir, "yolo_union_val"),
    output_yaml_path=os.path.join(base_dir, "yolo_union_dataset.yaml")
)


✅ YAML dosyası oluşturuldu: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_dataset.yaml


# YOLOv11 ile Model Eğitimi (Train + Val)

In [7]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt")

results = model.train(
    data=os.path.join(base_dir, "yolo_union_dataset.yaml"),
    epochs=100,
    imgsz=640,
    device="cpu"
)


100%|██████████| 5.35M/5.35M [00:00<00:00, 9.76MB/s]


New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.155  Python-3.9.1 torch-2.7.1+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64,

train: Scanning C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\labels... 881 images, 21 backgrounds, 10 corrupt: 100%|██████████| 881/881 [00:02<00:00, 300.66it/s]

train: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\images\056d2176-000087.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.6507      1.5216      1.9279]
train: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\images\174da6c5-20250506_144440.jpg: corrupt JPEG restored and saved
train: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\images\2f6ebdf2-000046.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.105      1.0759]
train: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\images\46bb4812-000137.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     4.9925      1.3188      2.8348      1.0442      1.1548]
train: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\images\61bb8487-000075.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.6817]
train: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\images\6497

train: New cache created: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.20.1 ms, read: 325.6334.7 MB/s, size: 956.8 KB)


...

    100/100         0G      0.692     0.8465      1.011         12        640: 100%|██████████| 55/55 [13:04<00:00, 14.27s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [01:43<00:00,  9.43s/it]

                   all        327       1114      0.869      0.661      0.725       0.59



100 epochs completed in 26.846 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train\weights\best.pt, 5.5MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.155  Python-3.9.1 torch-2.7.1+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)
YOLO11n summary (fused): 100 layers, 2,587,612 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [01:18<00:00,  7.18s/it]


                   all        327       1114       0.81      0.699      0.733      0.592
              barbecue         40         42      0.898      0.619      0.672      0.456
                 bench         13         15      0.535      0.333      0.353      0.275
               bicycle         24         29      0.844      0.749       0.77      0.612
                   car         15         19      0.885      0.579      0.637      0.545
              chainsaw         35         39      0.851      0.735       0.75      0.639
                 chair         83        246       0.71      0.577      0.597        0.4
                 drill         22         25      0.705       0.92      0.867      0.571
                ladder         12         14      0.854      0.837      0.861      0.755
                  lamp         14         44      0.434      0.205      0.215      0.154
                laptop         28         30      0.924      0.805      0.821      0.726
             lawnmowe

# Test veri kümesi üzerinde model tahmini (inference)

In [9]:
from ultralytics import YOLO
import os

base_dir = r"C:\Users\hp\Desktop\YOLOv11_union_100"  # 👈 kendi dizin yolunu buraya yazdın mı emin ol

model = YOLO("runs/detect/train/weights/best.pt")

results = model.predict(
    source=os.path.join(base_dir, "yolo_union_test/images"),
    save=True,
    save_txt=True,
    project=os.path.join(base_dir, "test_results"),
    name="predictions",
    conf=0.25
)

print("✅ Test tamamlandı. Sonuçlar:", os.path.join(base_dir, "test_results", "predictions"))



image 1/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\0034478e-000022.jpg: 640x640 1 leaf_blower, 320.6ms
image 2/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\00be4579-000082.jpg: 640x448 2 chairs, 1 table, 307.1ms
image 3/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\023bf6b7-59.webp: 640x480 1 motorcycle, 1036.0ms
image 4/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\034df7b0-20250531_2306_Garden_Ladder_View_simple_compose_01jwm2kbwfffvr9csqg26rb56m.png: 640x448 1 ladder, 281.8ms
image 5/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\036f1f53-1.png: 448x640 2 scooters, 264.0ms
image 6/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\03f1a1e8-ec0d4011-b139-4a9a-bca6-f25b823b2f76.jpeg: 640x448 1 ladder, 207.6ms
image 7/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\0418011c-000091.jpg: 576x640 1 toolbox, 266.0ms
image 8/341 C:\Users\hp\Desktop\YOLOv11_union_

# pickle elde etme

##  1. Pretrained model ile tahmin

In [10]:
from ultralytics import YOLO
import pickle

# Eğitilmiş modeli yeniden yükle
model = YOLO("runs/detect/train/weights/best.pt")  # trainX klasör adını kontrol et

# Eğitim sonucunu simüle etmek için boş bir sözlük oluştur (çünkü eski training sonuçları artık yok)
results = {"note": "Eğitim sonucu burada bulunamadı, sadece class_names kaydedildi."}

# Pickle dosyasını oluştur
data = {
    "class_names": model.names,
    "train_results": results
}

with open("training_summary.pkl", "wb") as f:
    pickle.dump(data, f)

print("✅ training_summary.pkl dosyası başarıyla oluşturuldu.")

print("✅ training_summary.pkl dosyası oluşturuldu.")


✅ training_summary.pkl dosyası başarıyla oluşturuldu.
✅ training_summary.pkl dosyası oluşturuldu.


In [1]:
from ultralytics import YOLO
import os

base_dir = r"C:\Users\hp\Desktop\YOLOv11_union_100"

# Pretrained (fine-tune edilmemiş) modeli yükle
pretrained_model = YOLO("yolo11n.pt")

# Test seti üzerinde tahmin yap
results_pretrained = pretrained_model.predict(
    source=os.path.join(base_dir, "yolo_union_test/images"),
    save=True,
    save_txt=True,
    project=os.path.join(base_dir, "test_results"),
    name="pretrained_predictions",
    conf=0.25
)

print("✅ Pretrained model ile test tamamlandı.")



image 1/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\0034478e-000022.jpg: 640x640 1 airplane, 1 scissors, 516.3ms
image 2/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\00be4579-000082.jpg: 640x448 1 dining table, 316.7ms
image 3/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\023bf6b7-59.webp: 640x480 1 person, 1 motorcycle, 639.2ms
image 4/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\034df7b0-20250531_2306_Garden_Ladder_View_simple_compose_01jwm2kbwfffvr9csqg26rb56m.png: 640x448 (no detections), 317.1ms
image 5/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\036f1f53-1.png: 448x640 (no detections), 297.6ms
image 6/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\03f1a1e8-ec0d4011-b139-4a9a-bca6-f25b823b2f76.jpeg: 640x448 (no detections), 251.0ms
image 7/341 C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\0418011c-000091.jpg: 576x640 1 suitcase, 416.9ms
image 8/341

In [7]:
!C:\Users\hp\AppData\Local\Programs\Python\Python39\python.exe -m pip install mean_average_precision

In [12]:
import os
import json
from tqdm import tqdm

def coco_to_yolo(coco_json_path, output_txt_dir, image_dir, img_ext=".jpg"):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # ID -> filename eşlemesi
    image_id_to_filename = {img['id']: img['file_name'] for img in coco_data['images']}
    image_sizes = {
        os.path.basename(img['file_name']): (img['width'], img['height'])
        for img in coco_data['images']
    }

    # Klasör yoksa oluştur
    os.makedirs(output_txt_dir, exist_ok=True)

    # Her görsel için boş bir .txt dosyası oluştur
    for full_filename in image_id_to_filename.values():
        filename = os.path.basename(full_filename)
        txt_name = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(output_txt_dir, txt_name)
        with open(txt_path, 'w') as f:
            pass  # başta boş bırakıyoruz

    # Her annotation satırını ilgili .txt dosyasına ekle
    for ann in tqdm(coco_data['annotations'], desc="Dönüştürülüyor"):
        image_id = ann['image_id']
        category_id = ann['category_id']
        bbox = ann['bbox']  # [x, y, width, height]

        full_filename = image_id_to_filename[image_id]
        filename = os.path.basename(full_filename)
        width, height = image_sizes[filename]

        # COCO → YOLO: normalize edilmiş x_center, y_center, w, h
        x, y, w, h = bbox
        x_center = (x + w / 2) / width
        y_center = (y + h / 2) / height
        w /= width
        h /= height

        txt_filename = os.path.splitext(filename)[0] + ".txt"
        txt_path = os.path.join(output_txt_dir, txt_filename)

        with open(txt_path, 'a') as f:
            f.write(f"{category_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")

    print(f"\n✅ {len(image_id_to_filename)} adet dosya oluşturuldu: {output_txt_dir}")

coco_to_yolo(
    coco_json_path=r"C:\Users\hp\Desktop\YOLOv11_union_100\merged_annotations_cleaned_test.json",
    output_txt_dir=r"C:\Users\hp\Desktop\YOLOv11_union_100\labels_gt",
    image_dir=r"C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images"
)



Dönüştürülüyor: 100%|██████████| 1159/1159 [00:02<00:00, 561.77it/s]


✅ 358 adet dosya oluşturuldu: C:\Users\hp\Desktop\YOLOv11_union_100\labels_gt


In [13]:
res1 = evaluate(
    gt_dir=r"C:\Users\hp\Desktop\YOLOv11_union_100\labels_gt",
    pred_dir=r"C:\Users\hp\Desktop\YOLOv11_union_100\test_results\pretrained_predictions\labels"
)

res2 = evaluate(
    gt_dir=r"C:\Users\hp\Desktop\YOLOv11_union_100\labels_gt",
    pred_dir=r"C:\Users\hp\Desktop\YOLOv11_union_100\test_results\predictions\labels"
)

print("Pretrained:", res1)
print("Fine-tuned:", res2)


Pretrained: {'TP': 336, 'GT': 924, 'PRED': 991, 'Precision': 0.3391, 'Recall': 0.3636, 'F1': 0.3509}
Fine-tuned: {'TP': 756, 'GT': 1090, 'PRED': 967, 'Precision': 0.7818, 'Recall': 0.6936, 'F1': 0.7351}


In [16]:
from ultralytics import YOLO
import os

base_dir = r"C:\Users\hp\Desktop\YOLOv11_union_100"
yaml_path = os.path.join(base_dir, "yolo_union_dataset.yaml")

pretrained = YOLO("yolo11n.pt")
print("\n▶ Pretrained model mAP hesaplanıyor...")
pretrained_metrics = pretrained.val(data=yaml_path, split='test')

finetuned = YOLO(os.path.join(base_dir, "runs", "detect", "train", "weights", "best.pt"))
print("\n▶ Fine-tuned model mAP hesaplanıyor...")
finetuned_metrics = finetuned.val(data=yaml_path, split='test')

print("\n📊 Pretrained mAP:")
print(f"mAP@0.5:      {pretrained_metrics.box.map50:.4f}")
print(f"mAP@0.5:95:   {pretrained_metrics.box.map:.4f}")

print("\n📊 Fine-tuned mAP:")
print(f"mAP@0.5:      {finetuned_metrics.box.map50:.4f}")
print(f"mAP@0.5:95:   {finetuned_metrics.box.map:.4f}")



▶ Pretrained model mAP hesaplanıyor...
Ultralytics 8.3.155  Python-3.9.1 torch-2.7.1+cpu CPU (Intel Core(TM) i5-6200U 2.30GHz)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs
val: Fast image access  (ping: 0.50.2 ms, read: 143.260.7 MB/s, size: 822.5 KB)


val: Scanning C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\labels... 341 images, 5 backgrounds, 2 corrupt: 100%|██████████| 341/341 [00:01<00:00, 197.24it/s]

val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\22d3a7b1-1000069747.jpg: corrupt JPEG restored and saved
val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\412ae48f-000032.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.6755      1.2999]
val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\5d3b488d-000109.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0862       1.222]
val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\e63dd75b-20250506_144401.jpg: corrupt JPEG restored and saved
val: New cache created: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\labels.cache



c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [02:16<00:00,  6.19s/it]


                   all        339       1076      0.277     0.0243     0.0137    0.00978
                person         31         36          0          0   0.000397   0.000193
               bicycle         15         21          0          0   0.000531   5.31e-05
                   car         19         27          0          0   0.000231   0.000185
            motorcycle         19         23     0.0318     0.0435    0.00666    0.00519
              airplane         34         43      0.093     0.0233     0.0176     0.0148
                   bus         73        221          0          0    0.00425    0.00194
                 train         19         21          0          0          0          0
                 truck         14         14          0          0    0.00748    0.00408
                  boat         15         44          0          0          0          0
         traffic light         22         23          0          0          0          0
          fire hydran

val: Scanning C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\labels.cache... 341 images, 5 backgrounds, 2 corrupt: 100%|██████████| 341/341 [00:00<?, ?it/s]

val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\22d3a7b1-1000069747.jpg: corrupt JPEG restored and saved
val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\412ae48f-000032.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.6755      1.2999]
val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\5d3b488d-000109.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0862       1.222]
val: C:\Users\hp\Desktop\YOLOv11_union_100\yolo_union_test\images\e63dd75b-20250506_144401.jpg: corrupt JPEG restored and saved



c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [01:45<00:00,  4.81s/it]


                   all        339       1076      0.888      0.687      0.757      0.626
              barbecue         31         36      0.924      0.556      0.694      0.499
                 bench         15         21       0.94      0.571      0.652      0.577
               bicycle         19         27      0.949      0.444      0.757      0.538
                   car         19         23          1      0.647       0.68      0.591
              chainsaw         34         43      0.843      0.791      0.855      0.741
                 chair         73        221       0.86      0.474      0.607      0.419
                 drill         19         21      0.838       0.81      0.847      0.706
                ladder         14         14      0.909          1      0.995      0.865
                  lamp         15         44      0.536      0.136      0.165      0.118
                laptop         22         23      0.905      0.826      0.834      0.779
             lawnmowe